In [ ]:

%tensorflow_version 2.x

!pip install simplejson
import simplejson as json
import tensorflow as tf
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Dense, Reshape, Activation, Dropout, Input,  LSTM, Embedding,Flatten,Bidirectional, TimeDistributed,RepeatVector,Permute,Concatenate,Multiply
from tensorflow.python.keras.models import load_model, Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import os
import multiprocessing
from google.colab import drive
from random import seed , choice
import string
import re
import io
import pandas as pd
drive.mount('/content/drive')
from keras.utils.generic_utils import get_custom_objects
#print(tf.__version__)
#tf.enable_eager_execution() #Because is not Tensorflow 2
#tf.compat.v1.disable_eager_execution()
import configparser #https://docs.python.org/3/library/configparser.html
import time

from sklearn.model_selection import train_test_split

import os


INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CollectiveCommunication.AUTO
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)




Running on TPU  ['10.52.252.178:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.52.252.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.52.252.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:

print(tf.version)
config = configparser.ConfigParser()
config = {'dsize': 10000,'batch_size': 16*tpu_strategy.num_replicas_in_sync, "emb_dim":300, "epochs":50,"sample_ignored" : 1 }

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:

"""***************************** Get Word Embeddings *****************************"""
def get_word_embeddings(tokenizer_eng,vocab_len):
  embeddings_matrix = np.zeros((vocab_len,config['emb_dim']))
  f= open("/content/drive/My Drive/Colab Notebooks/Embeddings/glove.6B/glove.6B."+str(config['emb_dim'])+"d.txt")

  for line in f:
      values = line.split()
      if(values[0] in tokenizer_eng.word_index):
          word = values[0]
          coefs = [float(i) for i in values[1:]]
          embeddings_matrix[tokenizer_eng.word_index[word],:] = coefs
  f.close()
  return embeddings_matrix


In [ ]:

"""***************************** Get Dataset - read file *****************************"""
def get_text():
  #http://www.datasciencemadesimple.com/strip-space-column-pandas-dataframe-leading-trailing-2/
  data = pd.DataFrame(columns=["ds_eng", "ds_pt"])
  data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Portfolio/Translator/en-pt.bicleaner07.txt',  names=["ds_eng", "ds_pt"],sep="\t", encoding='utf-8' ,nrows= config['dsize'])
  ind_eng=0
  ind_pt=0
  data.ds_eng =(("<start> "+((((data.ds_eng.str.replace('[^\w\s\d+]','')).str.strip()).str.lower()).astype(str))+" <end>")).sort_values(axis=0, ascending=True) 
  data.ds_pt = (("<start> "+((((data.ds_pt.str.replace('[^\w\s\d+]','')).str.strip()).str.lower()).astype(str))+" <end>")).sort_values(axis=0, ascending=True)
  
  return data


In [ ]:

"""***************************** Get Tokens *****************************"""
def get_tokenized_dataset(data):

  print("Train - Tokenizer get eng sentence: " +str(data.ds_eng.tolist()[100]))
  print("Train - Tokenizer get pt labels: "+ str(data.ds_pt.tolist()[100]))

  tokenizer_eng = Tokenizer(filters='',oov_token="<oov>")#filters: a string where each element is a character that will be filtered from the texts. The default is all punctuation, plus tabs and line breaks, minus the `'` character.
  tokenizer_pt = Tokenizer(filters='',oov_token="<oov>")# <oov> for unknown words

  tokenizer_eng.fit_on_texts(data.ds_eng) # Fit test
  tokenizer_pt.fit_on_texts(data.ds_pt)

  token_eng = tokenizer_eng.texts_to_sequences(data.ds_eng) # Convert text to token
  token_pt = tokenizer_pt.texts_to_sequences(data.ds_pt)

  print("tokenizer_eng : ",token_eng[100])
  print("tokenizer_pt : ",token_pt[100])

  token_eng = pad_sequences(token_eng,padding='post')
  token_pt = pad_sequences(token_pt,padding='post')
  
  return tokenizer_eng,tokenizer_pt,token_eng,token_pt



In [ ]:

text = get_text()

tokenizer_eng,tokenizer_token_pt, input_set,output_set  = get_tokenized_dataset(text)
train_input, val_input, train_output, val_output = train_test_split(input_set, output_set, test_size=0.1)

vocab_len_eng = len(tokenizer_eng.word_index) + 1
vocab_len_pt = len(tokenizer_token_pt.word_index) + 1

max_words_eng, max_words_pt = output_set.shape[1], input_set.shape[1]
embeddings_matrix = get_word_embeddings(tokenizer_eng,vocab_len_eng)

print("input.shape",input_set.shape)
print("output.shape",output_set.shape)
print("train_input.shape",train_input.shape)
print("train_output.shape",train_output.shape)
print("val_input.shape",val_input.shape)
print("val_output.shape",val_output.shape)

Train - Tokenizer get eng sentence: <start> censorship cuba discs information internet networks offline singularity totalitarianism <end>
Train - Tokenizer get pt labels: <start> censura cuba discos informações internet redes desligado singularidade totalitarismo <end>
tokenizer_eng :  [3, 319, 272, 5037, 83, 183, 492, 549, 651, 401, 4]
tokenizer_pt :  [2, 196, 175, 2277, 170, 150, 349, 435, 387, 230, 3]
input.shape (10000, 304)
output.shape (10000, 182)
train_input.shape (9000, 304)
train_output.shape (9000, 182)
val_input.shape (1000, 304)
val_output.shape (1000, 182)


In [ ]:
train_output_in=train_output[:,:(train_output.shape[1])-1]
train_output_out=train_output[:,1:(train_output.shape[1])]

dataset = tf.data.Dataset.from_tensor_slices((train_input,train_output)).shuffle(len(train_input))
dataset = dataset.batch(config['batch_size'], drop_remainder=True) # drop_remainder - drop the last batch if it's smaller than batch_size
example_input_batch, example_target_batch_ = next(iter(dataset))
example_input_batch.shape, example_target_batch_.shape 


(TensorShape([128, 304]), TensorShape([128, 182]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size,  enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = Embedding(vocab_size, config['emb_dim'], input_length=max_words_eng, weights=[embeddings_matrix],name ="emb_layer", trainable=False)
    self.lstm = LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state_h , state_c= self.lstm(x, initial_state = hidden)
    return output, state_h, state_c

  def initialize_hidden_state(self):
    return (tf.zeros([self.batch_sz, self.enc_units]),tf.zeros([self.batch_sz, self.enc_units]))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, s1,s2, values):
    # we are doing this to broadcast addition along the time axis to calculate the score
    query = tf.concat([s1,s2], axis=-1)
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden1,hidden2, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden1,hidden2, enc_output)
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state_h,state_c = self.lstm(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state_h,state_c, attention_weights

In [ ]:
units=256
with tpu_strategy.scope():
  # Encoder
  encoder = Encoder(vocab_len_eng,  units, config['batch_size'])
  sample_hidden= encoder.initialize_hidden_state()
  sample_output, sample_hidden, sample_c = encoder(example_input_batch, sample_hidden)
  print('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
  print('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))


  # Attention
  attention_layer = BahdanauAttention(10)
  attention_result, attention_weights = attention_layer(sample_hidden, sample_c , sample_output)

  print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
  print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

  # Decoder
  decoder = Decoder(vocab_len_pt, 1, units, config['batch_size'])
  sample_decoder_output,_, _, _ = decoder(tf.random.uniform((config['batch_size'], 1)),sample_hidden, sample_c, sample_output)
  print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

  optimizer = Adam(0.0004)
  loss_object = SparseCategoricalCrossentropy(from_logits=True, reduction='none')

  def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)



Encoder output shape: (batch size, sequence length, units) (128, 304, 256)
Encoder Hidden state shape: (batch size, units) (128, 256)
Attention result shape: (batch size, units) (128, 256)
Attention weights shape: (batch_size, sequence_length, 1) (128, 304, 1)
Decoder output shape: (batch_size, vocab size) (128, 28520)


In [ ]:
checkpoint_directory = "/content/drive/My Drive/Colab Notebooks/Portfolio/Translator/keras_model_default/"
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,encoder=encoder, decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):

  def _step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
      enc_output, enc_hidden,enc_c = encoder(inp, enc_hidden)
      
      dec_input = tf.expand_dims([tokenizer_token_pt.word_index['<start>']] * config['batch_size'], 1)
      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder

        predictions, enc_hidden, enc_c , _ = decoder(dec_input, enc_hidden,enc_c, enc_output)
        tar = tf.expand_dims(targ[:, t], 1)
        loss += loss_function(tar, predictions)
        
        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

  per_replica_losses = tpu_strategy.run(_step,args=(inp, targ, enc_hidden,))
  batch_loss_tep = tpu_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,axis=None)
  return batch_loss_tep

In [ ]:
EPOCHS = 1000
steps_per_epoch = len(train_input)//config['batch_size']
dataset = dataset.repeat(steps_per_epoch)

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    total_loss += train_step(inp, targ, enc_hidden)

  if epoch %20  ==0:
    print("Time taken for 1 epoch",(time.time() - start),"sec,  Epoch ",(epoch + 1)," -> Loss ",total_loss)





Time taken for 1 epoch 476.442444562912 sec,  Epoch  1  -> Loss  tf.Tensor(467.8415, shape=(), dtype=float32)
Time taken for 1 epoch 36.42127847671509 sec,  Epoch  21  -> Loss  tf.Tensor(247.01292, shape=(), dtype=float32)
Time taken for 1 epoch 36.4522430896759 sec,  Epoch  41  -> Loss  tf.Tensor(205.93108, shape=(), dtype=float32)
Time taken for 1 epoch 36.43180322647095 sec,  Epoch  61  -> Loss  tf.Tensor(175.57333, shape=(), dtype=float32)
Time taken for 1 epoch 36.41386389732361 sec,  Epoch  81  -> Loss  tf.Tensor(151.9371, shape=(), dtype=float32)
Time taken for 1 epoch 36.495097160339355 sec,  Epoch  101  -> Loss  tf.Tensor(130.46304, shape=(), dtype=float32)
Time taken for 1 epoch 36.43101692199707 sec,  Epoch  121  -> Loss  tf.Tensor(122.418274, shape=(), dtype=float32)
Time taken for 1 epoch 36.43678283691406 sec,  Epoch  141  -> Loss  tf.Tensor(105.70312, shape=(), dtype=float32)
Time taken for 1 epoch 36.45724391937256 sec,  Epoch  161  -> Loss  tf.Tensor(96.944305, shape=(

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_words_pt, max_words_eng))

  #sentence = preprocess_sentence(sentence)
  print(sentence.split(' '))
  
  inputs = [tokenizer_eng.word_index[i] for i in sentence.split(' ') if i != '']
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_words_eng,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units)),tf.zeros((1, units))]
  enc_out, enc_hidden,enc_c = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer_token_pt.word_index['<start>']], 0)

  for t in range(max_words_pt):
    predictions, dec_hidden,_, attention_weights = decoder(dec_input, dec_hidden,enc_c,enc_out) 

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    if predicted_id == 0:
      continue
      
    result += tokenizer_token_pt.index_word[predicted_id] + ' '

    if tokenizer_token_pt.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
print(text.ds_eng.tolist()[201])
print(text.ds_pt.tolist()[201])
translate("the world largest building")


In [ ]:
sentence = ['this','is','a','sentence']
sentence = ','.join(sentence)
print(sentence)